# How to make a pangenome with Anvi'o

## Set up the program and download your genomes 

Installing Anvio can be a pain, so see if you can get some help with it. Because it uses an interactive interface, downloading to Anthill may not be an option (regardless, it might be worth trying your luck). Otherwise, if using a Mac, Homebrew installation worked well for me… http://merenlab.org/2016/06/26/installation-v2/



In [ ]:
anvi-self-test --suite mini
 
#This should go through an example analysis and will eventually bring up the interactive interface
#If there is an issue, make sure that you have the correct format of Python running. For most of the issues there are blog posts on their slack or git accounts. For instance, my MacOSX had 2 bugs when upgrading to the new version, there are simple work arounds that are accessible. 

Once you have Anvio up and running, it is time to gather your genomes for analysis!

For this, go to NCBI Genome database. I searched for Pseudoalteromonas luteoviolacea sequenced genomes. https://www.ncbi.nlm.nih.gov/genome/12255?genome_assembly_id=283953

From here, locate the top box that says All **19 genomes** for species and and click ‘Download sequence and annotation from RefSeq’. This will bring you to a very plain webpage that contains the files to be downloaded for each of the strains that you are interested. Each of the links will be a folder which will bring you to different forms of the sequences. It is not a bad idea to also go back to the main genome page and click the Browse list link. This will generate a table with all of the samples available. This table can be downloaded to a CSV file by clicking the link in the top right corner.

We are interested in the files that are **GCF_########_genomic.fna.gz** This is a zipped file (.gz) with the Assembly ID, all genomic information in the form of a nucleotide fasta (.fna). Once downloaded, put them into a folder together using only letters, numbers, and _ (no spaces, dashes, or other special characters).

This brings up a good point, take this time to change the file names so that the only periods are associated with file extensions.Once your files are in your data folder, move your folder to the anvio directory (the place where Anvio was downloaded on your computer).

In [ ]:
#mv -r /path/to/[p.luteo_genomes] /usr/local/Cellar/Anvio/5.2
##this uses the move function to move your file (-r) 
#Anvio was installed in my computer to the second file path, but yours might be different. If you went through the installation step above, it will tell you where it was downloaded.

In [ ]:
cd /usr/local/Cellar/Anvio/5.2
#This utilizes the change directory command. In order to access the anvio commands, you have to be in the same folder as where they are kept 

## Pangenomic Analysis
The Anvio website has many different tutorials, features, and references. Follow that! http://merenlab.org/2016/11/08/pangenomics-v2/#summarizing-an-anvio-pan-genome

Okay so we know anvio works, but how about the pangenomic analysis? It’s a little different…


In [ ]:
anvi-self-test --suite pangenomics

Once this is up and running, now we can really start!

First, go ahead and unzip your genomes

In [ ]:
gunzip p.luteo_genomes/*.fna.gz
#this uses the gunzip command to unzip .gz files 
#It also utilizes the * wildcard value, which indicates 'run this command on any file name that ends in .fna.gz' 

Now take a look at one of your files to see what the contig names look like.

In [ ]:
head p.luteo_genomes/GCF_*.fna
#This shows you the first few lines of code for this file

If the contig names have any spaces, periods, dashes, or other special characters, we will need to rename them. There is an anvio code for this. If your contigs contain only letters, numbers, and underscores, disregard the code below. 

In [ ]:
for f in p.luteo_genomes/*.fna; do bin/anvi-script-reformat-fasta "$f" -o "$f"_fixed.fa --simplify-names -r p.luteo_genomes/names_report; done
#This code uses a for loop to perform the code seen after the blue 'do' iteratively over a folder of files. 
#HEADS UP! Your output files here need to have the (.fa) extension, or else they wont work otherwise.
#consider using the flag --report-file to get a TAB-delimited file to keep track of the old contig names, and the new ones. For some reason I couldnt get this to work. 

Also pay attention to (.) periods in the file names, if there is something like **GCA_002.1.Hi1.fna** you will need to find and replace all of the periods except the last one for the file extension. When reading the files in, Anvio will only read up to the first period. See the output below for an example of how to get around this.

In [ ]:
for f in p.luteo_genomes/*.fa; do bin/anvi-gen-contigs-database -f "$f" -o "${f/%.fa/.db}"; done
#This command makes the .db files needed to run the anvio analysis. Otherwise, you can use the shortcut script provided by anvio for the next few steps
#anvi-script-FASTA-to-contigs-db

#The output here is really important. It uses pattern recognition to extract the first part of the file name (ie the part that you want), %removes the original file format .fa, and replaces it with .db

In [ ]:
for f in p.luteo_genomes/*.db; do bin/anvi-run-hmms -c "$f" -T 4; done

#Run the hmm ribosomal RNA profiles on your samples
#See that -T indicates how many threads from your computer you are running. I dont do more than 4, but if you are running from Anthill, you can do more.
#Citation: Seemann T, https://github.com/tseemann/barrnap

After running the codes above, you should have 2/3 file types/sample: GCF_####.fa, GCF_####.db, GCF_####-MAP.txt <-- this one you only get if you run the anvi-script-FASTA-to-contigs-db code. 

Next let’s run ncbi’s COGs to attribute gene call to function. If you havent run this analysis before, use the code below:

In [ ]:
bin/anvi-setup-ncbi-cogs

Next, let’s run the NCBI cog gene caller. This analysis might take and hour or so, but not too long comparatively.

In [ ]:
for f in p.luteo_genomes/*.db; do bin/anvi-run-ncbi-cogs -c "$f" --cog-data-dir /usr/local/Cellar/Anvio/5.1/libexec/lib/python3.7/site-packages/anvio/data/misc/COG -T 4; done

#Because I set mine up when I was using a different version of Anvio, I have to send anvio to the cog directory, seen by the --cog-data-dir command above. 

Now we are finally ready to create an anvio genome storage file for your pangenomic analysis. This requires a single input: a .txt file (created in textedit is fine) that has two columns: name of your strain [tab] file directory(write the file directory with the thought of which directory you will be executing your code from). This is a very simple file, they have an example of one on their website if you are confused. Remember, Anvio hates special characters, so the same goes for your bacterial strain names.


In [ ]:
bin/anvi-gen-genomes-storage -e p.luteo_genomes/p.luteo_external_genomes -o p.luteo_genomes/p.luteo_all-GENOMES.db

#It is important here to remember file paths and inputs. I have found it easier to make the file path just the .db file name, and then adjust my -e input to the correct directory
#For some reason, you HAVE to end the file output with '-GENOMES.db'.

Now that our genomes are created into a genome storage file **-GENOMES.db**, it’s time to make the PAN.db file! This analysis took a few hours for me. I used the –num-threads 4 function to increase the computing power associated with the analysis. Be careful of this if you are running low on CPUs.

In [ ]:
bin/anvi-pan-genome -g p.luteo_genomes/p.luteo_all-GENOMES.db -n Psuedoalteromonas_luteoviolacea_All_Pangenome -o p.luteo_genomes/ -T 4 --minbit 0.5 --mcl-inflation 10 --use-ncbi-blast


===============================================
A kind warning from Anvio: If you publish results from this workflow, please do not forget to cite DIAMOND
(doi:10.1038/nmeth.3176), unless you use it with --use-ncbi-blast flag, and MCL
(http://micans.org/mcl/ and doi:10.1007/978-1-61779-361-5_15)

Anvi'o will use 'muscle' by Edgar, doi:10.1093/nar/gkh340
(http://www.drive5.com/muscle) to align your sequences.

….4 hours later…… You finally have all of the working parts to visualize! -GENOMES.db & -PAN.db. Now lets check this puppy out!

In [ ]:
bin/anvi-display-pan -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -g p.luteo_genomes/p.luteo_all-GENOMES.db

If you are experiencing issues getting the page to display, take a chunk of your error code and put it into the Github Issues search tab. I was getting an error that didnt let me open up the visualization after a MacOSX update. The error looked like this:

In [ ]:
objc[63488]: +[__NSPlaceholderDate initialize] may have been in progress in another thread when fork() was called.
objc[63488]: +[__NSPlaceholderDate initialize] may have been in progress in another thread when fork() was called. We cannot safely call it or ignore it in the fork() child process. Crashing instead. Set a breakpoint on objc_initializeAfterForkError to debug.

In [ ]:
#To fix the fork error:
export OBJC_DISABLE_INITIALIZE_FORK_SAFETY=YES
#Rerun the display pangenome code and it should work fine this time

By this point, you should be able to run the code and get the chrome popup that is the interactive webpage. Here, you can click around and change things --AWESOME!

PS if your plot isnt coming up, dont forget to click DRAW


## Manually binning genes with no known function

Now on to some more refined analyses... I have my MACs sequences that do not come up as cogs, so how do I find them and manually bin them?

In [ ]:
bin/anvi-script-add-default-collection -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db
#This makes a collection that includes EVERYTHING
#This will allow you to summarize everything and get the .fa sequences in the next couple of commands 

When you are finishing up, it’s not a bad idea to create a summary page. To do this, you need to interupt the interactive interface **CTRL+C**.

In [ ]:
bin/anvi-summarize -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -g p.luteo_genomes/p.luteo_all-GENOMES.db -C DEFAULT -o p.luteo_genomes/pan_all_DEFAULT_output --report-aa-seqs-for-gene-calls

In [ ]:
bin/anvi-get-sequences-for-gene-clusters -g p.luteo_genomes/p.luteo_all-GENOMES.db -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db --collection-name DEFAULT --bin-id EVERYTHING -o p.luteo_genomes/p.luteo_all_gene_clusters.fa

#This command creates the .fa files needed to search the entire pangenome for MACs. 

From here, you can put the .fa file into Blast on the command line

I generally use Anthill for this... The basics are as follows: 

Make a blast database file using: 
makeblastdb 

Then perform your blast using whichever flavor you like 
blastn, tblastn, blastx, etc. Just make sure that your inputs match the service you are using. At this point your .fa files have AA sequences for the CDS. My query is with nucleotides, so I performed a blastx. 

Always read your output formats -outfmt and write it out to a new file -o

From here, I had a list bin numbers that I could manually search through and then find using the Search by expression tab on the interactive interface. 

## Add a phylogeny to your display

Open up your interactive display again and take a look at the SCG genes. Dont forget to sort your gene clusters on the main tab either by SCG or Frequency. 

Select the SCG genes, and add them to a bin. To do this, click the dendrogram on the inner phylogeny to select all and add to a bin. I also selected some conserved gene clusters just on the outside of the SCGs to allow for more discreet separation of very closely related genomes. 

In [ ]:
bin/anvi-get-sequences-for-gene-clusters -g p.luteo_genomes/p.luteo_all-GENOMES.db -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db --collection-name SCGs --bin-id SCGs --concatenate-gene-clusters --max-num-genes-from-each-genome 1 -o p.luteo_genomes/p.luteo_all_SCGs_gene_clusters.fa
#This command is used with the bin created for the phylogeny
#use the --concatenate-gene-clusters command to create an aligned file of concatenated sequences

Download the aligned .fa file to create the phylogeny. Anvio has an in-house phylogeny maker 'FastTree', but it's not the best and it was throwing errors about improper tree formatting. Instead, I dragged my file into SEAVIEW. 
Here the alignment opened up and then you click.. 

File > Save as > Phylip

Then open up PhyML on ATGC
http://www.atgc-montpellier.fr/phyml/

Submit the job using the default parameters, taking into consideration the type of data that you have 

Citation: "SMS: Smart Model Selection in PhyML."
Vincent Lefort, Jean-Emmanuel Longueville, Olivier Gascuel.
Molecular Biology and Evolution, msx149, 2017.
https://academic.oup.com/mbe/article/34/9/2422/3788860

Once you get the results, download the folder. Click Tree Visualizsation and then export the tree using the dropdown tab on the top. Save as Newick file.

Now you can use this file to either upload it and manipulate it further in itol, or upload it directly to Anvio. 

In [ ]:
bin/anvi-import-misc-data -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -t layer_orders p.luteo_genomes/layer_orders.txt
#first you need to add the layer of the phylogeny. To do this make a tab delimited text file 

In [ ]:
anvi-display-pan -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -g p.luteo_genomes/p.luteo_all-GENOMES.db -t p.luteo_genomes/phylogeny_pluteo
#The pangenome can be displayed using the normal command, plus adding the option -t with the phylogeny. 

## Compute Average Nucleotide Index

Citation: Pritchard et al. (DOI: 10.1039/C5AY02550H)

In [ ]:
bin/anvi-compute-ani -e p.luteo_genomes/p.luteo_external_genomes -o p.luteo_genomes/ani -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -T 4
#this one takes a while. 

In [ ]:
bin/anvi-display-pan -g p.luteo_genomes/p.luteo_all-GENOMES.db -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db
#Display one more time with all of your features. 

Don't forget to save each view as a state, so that you can come back to the same views with layers such as the ANI, and phylogeny. 

## Functional Analysis  

Add a catagorical layer for clade, to delineate clustering of groups. The file created "layer-additional-data.txt" is a tab delimited file that has the strain names separated by the clade group. In this I characterized clade 1 as A, and clade 2 as B. 

Check out this page if the layers and tables are confusing 
http://merenlab.org/2017/12/11/additional-data-tables/

In [ ]:
bin/anvi-import-misc-data p.luteo_genomes/layer-additional-data.txt -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db --target-data-table layers

Feel free to visualize the pangenome here to see if it incorporated your new layers. To view, click the layers tab. 

In [ ]:
bin/anvi-display-pan -g p.luteo_genomes/p.luteo_all-GENOMES.db -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db
#Change the colors of the layers by clicking the layers tab and changing the color key
#When done Ctrl + C to terminate the window

Download a file based on the catagorical layer you just added (Clade in this case), that gives you the enriched functions.

In [ ]:
bin/anvi-get-enriched-functions-per-pan-group -p p.luteo_genomes/Psuedoalteromonas_luteoviolacea_All_Pangenome-PAN.db -g p.luteo_genomes/p.luteo_all-GENOMES.db --category-variable clade --annotation-source COG_FUNCTION -o Pluteo-Pan-enriched-functions-clade

This analysis produced a Tab-delimited file with hits for every CDS with known COG function. I brought this table into R and wrote a code to filter the table based on significantly regulated genes. Code, R Markdown file, and outputs can be found on Github at: https://github.com/Aalker/research-codes